### Scrape  transcript url by ticler

In [ ]:
import asyncio
from playwright.async_api import async_playwright
import time
import json
import random
import os

import nest_asyncio
nest_asyncio.apply()

# 模拟滚动获取链接
async def scroll_page_until_count_stops(page):
    prev_article_count = 0
    same_count = 0
    max_same = 3 

    while True:
        await page.mouse.wheel(0, random.randint(1500, 4000))
        await asyncio.sleep(random.uniform(0.8, 1.5))

        article_links = await page.query_selector_all('h3 a[data-test-id="post-list-item-title"]')
        curr_article_count = len(article_links)

        if curr_article_count > prev_article_count:
            print(f"✅ 已加载 {curr_article_count} 个文章链接...")
            prev_article_count = curr_article_count
            same_count = 0
        else:
            same_count += 1
            if same_count >= max_same:
                print("🛑 连续多次未加载新链接，停止滚动。")
                break
# 人工验证
async def wait_for_captcha_and_resume(page):
    """
    检测验证码页面，如果存在，则暂停程序等待用户手动解决。
    """
    print("🤖 正在检查是否存在验证码...")
    try:
        # 使用文本内容精确定位“按住”按钮，并设置一个短超时
        captcha_button = page.get_by_text("按住").first
        await captcha_button.wait_for(state="visible", timeout=5000)
        
        # 如果按钮可见，说明验证码已出现
        print("\n" + "="*50)
        print("⚠️ 检测到验证码！请在浏览器中手动完成验证。")
        print("💡 完成后，请回到此终端并按 Enter 键继续。")
        print("="*50 + "\n")
        
        # 暂停程序，等待用户输入
        input("按下 Enter 键以继续...")
        
        # 验证码解决后，等待页面加载完成
        await page.wait_for_load_state('domcontentloaded')
        print("✅ 验证码已解决，程序继续。")
        
    except Exception:
        # 如果在超时时间内没有找到验证码，则认为不存在，直接继续
        print("🚀 未检测到验证码，继续执行。")
        pass

# 进度于异常记录
def get_completed_tickers(progress_file="progress_second.txt"):
    """从进度文件中读取已完成的 ticker"""
    if os.path.exists(progress_file):
        with open(progress_file, "r") as f:
            return set(line.strip() for line in f)
    return set()

def mark_as_completed(ticker, progress_file="progress_second.txt"):
    """将已完成的 ticker 写入进度文件"""
    with open(progress_file, "a") as f:
        f.write(f"{ticker}\n")

def mark_as_excluded(ticker, exclude_file="exclude_transcript_second.txt"):
    """将没有 transcripts 的 ticker 写入单独文件"""
    if os.path.exists(exclude_file):
        with open(exclude_file, "a") as f:
            f.write(f"{ticker}\n")
    else:
        with open(exclude_file, "w") as f:
            f.write(f"{ticker}\n")

def mark_as_failed(ticker, failed_file="failed_second.txt"):
    """将爬取失败的 ticker 写入单独文件"""
    if os.path.exists(failed_file):
        with open(failed_file, "a") as f:
            f.write(f"{ticker}\n")
    else:
        with open(failed_file, "w") as f:
            f.write(f"{ticker}\n")

# 主流程
async def connect_and_scrape(ticker_list):
    print("🚀 正在连接 Chrome...")
    async with async_playwright() as p:
        try:
            browser = await p.chromium.connect_over_cdp("http://localhost:9333")
            context = browser.contexts[0]
        except Exception as e:
            print(f"❌ 连接失败: {e}")
            return

        completed_tickers = get_completed_tickers()
        excluded_tickers = get_completed_tickers(progress_file="exclude_transcript_second.txt")
        # 新增读取失败的 ticker 列表
        failed_tickers = get_completed_tickers(progress_file="failed_second.txt")
        
        # 综合考虑所有已处理的 ticker
        all_processed_tickers = completed_tickers.union(excluded_tickers).union(failed_tickers)
        tickers_to_process = [t for t in ticker_list if t not in all_processed_tickers]

        if not tickers_to_process:
            print("🎉 所有 tickers 任务已完成、已排除或已失败，无需再次运行。")
            return

        print(f"📄 将从 {len(tickers_to_process)} 个未完成的 tickers 开始爬取...")

        output_folder = "transcripts_second"
        if not os.path.exists(output_folder):
            os.makedirs(output_folder)
            print(f"📁 已创建输出文件夹: {output_folder}")
        
        # 确保排除和失败文件存在
        exclude_file = "exclude_transcript_second.txt"
        if not os.path.exists(exclude_file):
            with open(exclude_file, "w") as f:
                pass
        failed_file = "failed_second.txt"
        if not os.path.exists(failed_file):
            with open(failed_file, "w") as f:
                pass

        for ticker in tickers_to_process:
            url = f'https://seekingalpha.com/symbol/{ticker}/earnings/transcripts'
            
            try:
                page = None
                pages = context.pages
                for existing_page in pages:
                    if "seekingalpha.com" in existing_page.url:
                        page = existing_page
                        break
                
                if not page:
                    page = await context.new_page()

                print(f"\n➡️ 正在处理 {ticker}...")
                print(f"➡️ 导航到 {url}")
                await page.goto(url, timeout=60000)
                
                await wait_for_captcha_and_resume(page)

                print("🔍 检查 'Transcripts & Insights' 按钮是否存在...")
                transcripts_button = await page.query_selector('a[data-test-id="earnings/transcripts"]')
                
                if not transcripts_button:
                    print(f"⚠️ {ticker} 页面不包含 'Transcripts & Insights' 按钮。")
                    print(f"📄 为 {ticker} 创建一个空的 JSON 文件并标记为已排除。")
                    
                    results = []
                    file_path = os.path.join(output_folder, f"{ticker}.json")
                    with open(file_path, "w", encoding='utf-8') as f:
                        json.dump(results, f, indent=2, ensure_ascii=False)
                    
                    mark_as_excluded(ticker)
                    print(f"✅ {ticker} 数据已成功保存至 {file_path} (无内容)。")
                    continue
                
                print("✅ 按钮存在，开始爬取。")

                print("⌛ 等待页面加载文章内容...")
                await page.wait_for_selector('h3 a[data-test-id="post-list-item-title"]', timeout=30000)
                print("✅ 页面加载完毕。")

                print("🚀 开始自动滚动页面加载更多内容...")
                await scroll_page_until_count_stops(page)

                article_links = await page.query_selector_all('h3 a[data-test-id="post-list-item-title"]')
                print(f"📄 共找到 {len(article_links)} 个文章链接 for {ticker}")

                results = []
                for link_elem in article_links:
                    href = await link_elem.get_attribute('href')
                    title = await link_elem.inner_text()
                    
                    if href:
                        clean_href = href.split('#')[0]
                        results.append({
                            "title": title.strip(),
                            "link": f"https://seekingalpha.com{clean_href}"
                        })

                print(f"📄 最终处理后找到 {len(results)} 个链接")
                
                file_path = os.path.join(output_folder, f"{ticker}.json")
                with open(file_path, "w", encoding='utf-8') as f:
                    json.dump(results, f, indent=2, ensure_ascii=False)

                mark_as_completed(ticker)
                print(f"✅ {ticker} 数据已成功保存至 {file_path}")

            except Exception as e:
                print(f"❌ 抓取 {ticker} 失败: {e}")
                print(f"⚠️ 错误信息: {e}")
                mark_as_failed(ticker)
                
        await browser.close()
        print("✅ 所有任务完成，浏览器已关闭。")

# 假设你已有一个 ticker_list，例如：
# ticker_list = ["AAPL", "MSFT", "NVDA", ...]
# 实际上这里的ticker_list已经放在 github 仓库中，是SeekingAlpha截止2025年9月份所有可获取的企业代码的codes.json文件
# 需要将codes.json文件导入后运行
asyncio.run(connect_and_scrape(ticker_list))